In [1]:
import os
import glob
import re
import datetime
import glob
from datetime import date, time, timedelta
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain
from cv2 import VideoCapture, CAP_PROP_FRAME_COUNT, CAP_PROP_FPS, CAP_PROP_POS_FRAMES
import ffmpeg
from imutils.video import FileVideoStream
import time
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, export_png, export_svg
from bokeh.models import Span, DatetimeTicker, DatetimeTickFormatter
from bokeh.layouts import column, gridplot
from tsmoothie.smoother import *
from utils import load_diffs, load_slide_changes, load_intervals, filter_video

sns.set()
output_notebook()

Loading BokehJS ...

# Paths

In [2]:
#meeting_id = 160320
#meeting_id = 220120
meeting_id = 170127
#meeting_id = 83512718053

In [3]:
masked = True

In [4]:
video_path = glob.glob(f'zoom_data/{meeting_id}/*.mp4')[0]
print(video_path)
if masked is None:
    diff_path = f'diff_data/diffs_{meeting_id}_cossim.csv'
    diff_path_masked = f'diff_data/diffs_{meeting_id}_pct_masked_cossim.csv'
elif masked:
    diff_path = f'diff_data/diffs_{meeting_id}_pct_masked_cossim.csv'
else:
    diff_path = f'diff_data/diffs_{meeting_id}_cossim.csv'
sc_labels = f'slide_change_labels/{meeting_id}.csv'
interval_path = 'interval_data/intervals.csv'

zoom_data/170127/GMT20210614-170127_Recording_2350x1322.mp4


# Load FPS, Diffs, and Slide Change Labels

In [5]:
vidcap = VideoCapture(video_path)
fps = vidcap.get(CAP_PROP_FPS)
ddiffs = load_diffs(diff_path, fps)
sldf2 = load_slide_changes(sc_labels)
idf = load_intervals(interval_path, meeting_id=meeting_id)

# Visualize

In [7]:
ddiffs = filter_video(ddiffs, idf)
ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
ddiffs_time = ddiffs.elapsed_dt

In [7]:
ddiffs

,meeting_id,elapsed_dt,l1_diff,l2_diff,cos_sim_diff
0,170127,1970-01-01 00:00:01,0.0,0.000000,0.000000e+00
1,170127,1970-01-01 00:00:02,13160402.0,57322.628935,4.460693e-05
2,170127,1970-01-01 00:00:03,1877100.0,3875.137626,0.000000e+00
3,170127,1970-01-01 00:00:04,0.0,0.000000,0.000000e+00
4,170127,1970-01-01 00:00:05,3760052.0,30732.921989,6.226180e-07
...,...,...,...,...,...
6142,170127,1970-01-01 01:58:54,0.0,0.000000,8.013371e-06
6143,170127,1970-01-01 01:58:55,0.0,0.000000,4.367831e-07
6144,170127,1970-01-01 01:58:56,0.0,0.000000,0.000000e+00
6145,170127,1970-01-01 01:58:57,882961282.0,442324.399365,5.974587e-02


In [8]:
sldf2

,change_time,note,change_time_dt
0,0:00:57,NaN,1970-01-01 00:00:57
1,0:1:29,NaN,1970-01-01 00:01:29
2,0:2:07,NaN,1970-01-01 00:02:07
3,0:3:35,NaN,1970-01-01 00:03:35
4,0:5:37,NaN,1970-01-01 00:05:37
...,...,...,...
69,1:53:17,NaN,1970-01-01 01:53:17
70,1:54:41,NaN,1970-01-01 01:54:41
71,1:55:28,NaN,1970-01-01 01:55:28
72,1:57:08,NaN,1970-01-01 01:57:08


In [8]:
meetings = [160320, 220120, 170127, 83512718053]
#meetings = [170127]
masked = True

In [9]:
dd_list = []
for meeting_id in meetings:
    video_path = glob.glob(f'zoom_data/{meeting_id}/*.mp4')[0]
    print(video_path)
    if masked is None:
        diff_path = f'diff_data/diffs_{meeting_id}_cossim.csv'
        diff_path_masked = f'diff_data/diffs_{meeting_id}_pct_masked_cossim.csv'
    elif masked:
        diff_path = f'diff_data/diffs_{meeting_id}_pct_masked_cossim.csv'
    else:
        diff_path = f'diff_data/diffs_{meeting_id}_cossim.csv'
    sc_labels = f'slide_change_labels/{meeting_id}.csv'
    interval_path = 'interval_data/intervals.csv'
    
    vidcap = VideoCapture(video_path)
    fps = vidcap.get(CAP_PROP_FPS)
    ddiffs = load_diffs(diff_path, fps)
    sldf = load_slide_changes(sc_labels)
    idf = load_intervals(interval_path, meeting_id=meeting_id)
    
    ddiffs = filter_video(ddiffs, idf)
    dd_list.append(ddiffs)
ddiffs_df = pd.concat(dd_list, ignore_index=True)

zoom_data/160320/GMT20210614-160320_Recording_2020x1380.mp4
zoom_data/220120/GMT20210614-220120_Recording_1920x1170.mp4
zoom_data/170127/GMT20210614-170127_Recording_2350x1322.mp4
zoom_data/83512718053/GMT20210527-002121_Recording_1920x1200.mp4


In [5]:
sldf

,change_time,note,change_time_dt
0,0:8:20,NaN,1970-01-01 00:08:20
1,0:9:29,NaN,1970-01-01 00:09:29
2,0:9:35,NaN,1970-01-01 00:09:35
3,0:10:23,NaN,1970-01-01 00:10:23
4,0:10:58,NaN,1970-01-01 00:10:58
5,0:11:37,NaN,1970-01-01 00:11:37
6,0:11:44,NaN,1970-01-01 00:11:44
7,0:12:32,NaN,1970-01-01 00:12:32
8,0:14:44,NaN,1970-01-01 00:14:44
9,0:16:16,NaN,1970-01-01 00:16:16


### Stdev band

20/4.22 ~ decent  
20/5 ~ bad  
10/2.84 ~ decent

### For the 10/2.84 case

In [44]:
window = 10
num_stdevs = 2.84

#### Plot

In [45]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    ddiffs['rmean'] = ddiffs.cos_sim_diff.rolling(window).mean()
    ddiffs['band'] = ddiffs['rmean'] + ddiffs.cos_sim_diff.rolling(window).std() * num_stdevs
    ddiffs['slide_change'] = ddiffs.cos_sim_diff > ddiffs.band
    signals = ddiffs.query("slide_change")

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    #p.line(ddiffs_time, ddiffs.rmean, line_width=2, line_alpha=0.25, line_color='black')
    p.line(ddiffs_time, ddiffs.band, line_width=2, line_alpha=0.25, line_color='green')
    p.circle(signals.elapsed_dt, signals.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '14pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"

    plots.append(p)
plots[-1].xaxis.axis_label = "Elapsed Time"
plots[-1].xaxis.axis_label_text_font_size = "14pt"
col = column(*plots)
show(col)

#### Save

In [36]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    ddiffs['rmean'] = ddiffs.cos_sim_diff.rolling(window).mean()
    ddiffs['band'] = ddiffs['rmean'] + ddiffs.cos_sim_diff.rolling(window).std() * num_stdevs
    ddiffs['slide_change'] = ddiffs.cos_sim_diff > ddiffs.band
    signals = ddiffs.query("slide_change")

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    #p.line(ddiffs_time, ddiffs.rmean, line_width=2, line_alpha=0.25, line_color='black')
    p.line(ddiffs_time, ddiffs.band, line_width=2, line_alpha=0.25, line_color='green')
    p.circle(signals.elapsed_dt, signals.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '14pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"
    p.output_backend = "svg"

    plots.append({'meeting': meeting, 'plot': p})
plots[-1]['plot'].xaxis.axis_label = "Elapsed Time"
plots[-1]['plot'].xaxis.axis_label_text_font_size = "14pt"

for pdict in plots:
    export_svg(pdict['plot'], filename=f"thesis/mscs_thesis/figures/3_2_3_ma_10_284_{pdict['meeting']}.svg")

### For the 20/4.22 case

In [10]:
window = 20
num_stdevs = 4.22

#### Plot

In [12]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    ddiffs['rmean'] = ddiffs.cos_sim_diff.rolling(window).mean()
    ddiffs['band'] = ddiffs['rmean'] + ddiffs.cos_sim_diff.rolling(window).std() * num_stdevs
    ddiffs['slide_change'] = ddiffs.cos_sim_diff > ddiffs.band
    signals = ddiffs.query("slide_change")

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    #p.line(ddiffs_time, ddiffs.rmean, line_width=2, line_alpha=0.25, line_color='black')
    p.line(ddiffs_time, ddiffs.band, line_width=2, line_alpha=0.25, line_color='green')
    p.circle(signals.elapsed_dt, signals.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '14pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"
    p.output_backend = "svg"

    plots.append(p)
#plots[-1].xaxis.axis_label = "Elapsed Time"
#plots[-1].xaxis.axis_label_text_font_size = "14pt"
col = column(*plots)
show(col)

#### Save

In [39]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    ddiffs['rmean'] = ddiffs.cos_sim_diff.rolling(window).mean()
    ddiffs['band'] = ddiffs['rmean'] + ddiffs.cos_sim_diff.rolling(window).std() * num_stdevs
    ddiffs['slide_change'] = ddiffs.cos_sim_diff > ddiffs.band
    signals = ddiffs.query("slide_change")

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    #p.line(ddiffs_time, ddiffs.rmean, line_width=2, line_alpha=0.25, line_color='black')
    p.line(ddiffs_time, ddiffs.band, line_width=2, line_alpha=0.25, line_color='green')
    p.circle(signals.elapsed_dt, signals.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '14pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"
    p.output_backend = "svg"

    plots.append({'meeting': meeting, 'plot': p})
#plots[-1]['plot'].xaxis.axis_label = "Elapsed Time"
#plots[-1]['plot'].xaxis.axis_label_text_font_size = "14pt"

for pdict in plots:
    export_svg(pdict['plot'], filename=f"thesis/mscs_thesis/figures/3_2_3_ma_20_422_{pdict['meeting']}.svg")

### Kalman Filter

0.5/2 ~ so-so  
0 ~ decent  
Explain this

### For the 0.5/2 sig case

In [14]:
noise_level = 0.5
sig = 2

#### Plot

In [15]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    smoother = KalmanSmoother(component='level'
                             ,component_noise={'level': noise_level}
                             )
    smoother.smooth(ddiffs_cos)
    low, up = smoother.get_intervals('sigma_interval', n_sigma=sig)
    #low, up = smoother.get_intervals('kalman_interval', confidence=confidence)
    signals2 = ddiffs.loc[ddiffs.cos_sim_diff >= up.ravel(), :]

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    p.line(ddiffs_time, up.ravel(), line_width=2, line_alpha=0.5, line_color='black')
    p.circle(signals2.elapsed_dt, signals2.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '13pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"
    plots.append(p)
#plots[-1].xaxis.axis_label = "Elapsed Time"
#plots[-1].xaxis.axis_label_text_font_size = "14pt"
show(column(*plots))

#### Save

In [16]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    smoother = KalmanSmoother(component='level'
                             ,component_noise={'level': noise_level}
                             )
    smoother.smooth(ddiffs_cos)
    low, up = smoother.get_intervals('sigma_interval', n_sigma=sig)
    #low, up = smoother.get_intervals('kalman_interval', confidence=confidence)
    signals2 = ddiffs.loc[ddiffs.cos_sim_diff >= up.ravel(), :]

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    p.line(ddiffs_time, up.ravel(), line_width=2, line_alpha=0.5, line_color='black')
    p.circle(signals2.elapsed_dt, signals2.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '13pt'
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"
    p.output_backend = "svg"
    plots.append({'meeting': meeting, 'plot': p})
    
#plots[-1]['plot'].xaxis.axis_label = "Elapsed Time"
#plots[-1]['plot'].xaxis.axis_label_text_font_size = "14pt"
for pdict in plots:
    export_svg(pdict['plot'], filename=f"thesis/mscs_thesis/figures/3_2_4_kf_05_2_{pdict['meeting']}.svg")

### For the 0.5/1 sig case

In [19]:
noise_level = 0.5
sig = 1

#### Plot

In [18]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    smoother = KalmanSmoother(component='level'
                             ,component_noise={'level': noise_level}
                             )
    smoother.smooth(ddiffs_cos)
    low, up = smoother.get_intervals('sigma_interval', n_sigma=sig)
    #low, up = smoother.get_intervals('kalman_interval', confidence=confidence)
    signals2 = ddiffs.loc[ddiffs.cos_sim_diff >= up.ravel(), :]

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    p.line(ddiffs_time, up.ravel(), line_width=2, line_alpha=0.5, line_color='black')
    p.circle(signals2.elapsed_dt, signals2.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '13pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"
    plots.append(p)
show(column(*plots))

#### Save

In [20]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    smoother = KalmanSmoother(component='level'
                             ,component_noise={'level': noise_level}
                             )
    smoother.smooth(ddiffs_cos)
    low, up = smoother.get_intervals('sigma_interval', n_sigma=sig)
    #low, up = smoother.get_intervals('kalman_interval', confidence=confidence)
    signals2 = ddiffs.loc[ddiffs.cos_sim_diff >= up.ravel(), :]

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    p.line(ddiffs_time, up.ravel(), line_width=2, line_alpha=0.5, line_color='black')
    p.circle(signals2.elapsed_dt, signals2.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '13pt'
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "14pt"
    p.xaxis.axis_label = "Elapsed Time"
    p.xaxis.axis_label_text_font_size = "14pt"
    p.output_backend = "svg"
    plots.append({'meeting': meeting, 'plot': p})
for pdict in plots:
    export_svg(pdict['plot'], filename=f"thesis/mscs_thesis/figures/3_2_4_kf_05_1_{pdict['meeting']}.svg")

### For the 0.5/2 sig case vs the 0.5/0.1 case

In [65]:
noise_level = 0.5
sig = 2

In [66]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    smoother = KalmanSmoother(component='level'
                             ,component_noise={'level': noise_level}
                             )
    smoother.smooth(ddiffs_cos)
    low, up = smoother.get_intervals('sigma_interval', n_sigma=sig)
    #low, up = smoother.get_intervals('kalman_interval', confidence=confidence)
    signals2 = ddiffs.loc[ddiffs.cos_sim_diff >= up.ravel(), :]

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    p.line(ddiffs_time, up.ravel(), line_width=2, line_alpha=0.5, line_color='black')
    p.circle(signals2.elapsed_dt, signals2.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '13pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "13pt"
    p.output_backend = "svg"
    plots.append(p)
plots[-1].xaxis.axis_label = "Elapsed Time"
plots[-1].xaxis.axis_label_text_font_size = "13pt"
show(column(*plots))

In [67]:
noise_level = 0.5
sig = 0.1

In [68]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    smoother = KalmanSmoother(component='level'
                             ,component_noise={'level': noise_level}
                             )
    smoother.smooth(ddiffs_cos)
    low, up = smoother.get_intervals('sigma_interval', n_sigma=sig)
    #low, up = smoother.get_intervals('kalman_interval', confidence=confidence)
    signals2 = ddiffs.loc[ddiffs.cos_sim_diff >= up.ravel(), :]

    p = figure(plot_width=1000, plot_height=300, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    p.line(ddiffs_time, up.ravel(), line_width=2, line_alpha=0.5, line_color='black')
    p.circle(signals2.elapsed_dt, signals2.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '13pt'
    #p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    #p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "13pt"
    p.output_backend = "svg"
    plots.append(p)
plots[-1].xaxis.axis_label = "Elapsed Time"
plots[-1].xaxis.axis_label_text_font_size = "13pt"
show(column(*plots))

### Gaussian smoothing

knots/sig  
1/1 ~ decent  
10/1 ~ more errors

In [214]:
n_knots = 50
sigma = 1

In [215]:
plots = []
for meeting, ddiffs in ddiffs_df.groupby('meeting_id'):
    ddiffs_cos = ddiffs.cos_sim_diff.to_numpy()
    ddiffs_time = ddiffs.elapsed_dt
    #smoother = GaussianSmoother(n_knots=100, sigma=1)
    smoother = GaussianSmoother(n_knots=n_knots, sigma=sigma)
    smoother.smooth(ddiffs_cos)
    low, up = smoother.get_intervals('confidence_interval')
    #low, up = smoother.get_intervals('confidence_interval', n_sigma=2)
    signals2 = ddiffs.loc[ddiffs.cos_sim_diff >= up.ravel(), :]

    p = figure(plot_width=1200, plot_height=230, x_axis_type='datetime')
    p.line(ddiffs_time, ddiffs_cos, line_width=2, line_alpha=0.5)
    p.line(ddiffs_time, up.ravel(), line_width=2, line_alpha=0.5, line_color='black')
    p.circle(signals2.elapsed_dt, signals2.cos_sim_diff, alpha=0.25, color='red')

    sc_labels = f'slide_change_labels/{meeting}.csv'
    sldf = load_slide_changes(sc_labels)
    if sldf is not None:
        for sc in sldf.change_time_dt:
            c = Span(location=sc, dimension='height', line_color='black', line_width=0.5)
            p.add_layout(c)

    p.title.text = f"Meeting {meeting}"
    p.title.text_font_size = '13pt'
    p.xaxis.ticker = DatetimeTicker(desired_num_ticks=30, num_minor_ticks=10)
    p.xaxis.formatter = DatetimeTickFormatter(minutes='%M', seconds='%M:%S')
    p.xaxis.major_label_orientation = "vertical"
    p.xaxis.major_label_text_color = None
    p.xaxis.major_tick_line_color = None
    p.yaxis.axis_label = "Cosine distance (diff)"
    p.yaxis.axis_label_text_font_size = "13pt"
    plots.append(p)
plots[-1].xaxis.axis_label = "Elapsed Time"
plots[-1].xaxis.axis_label_text_font_size = "13pt"
show(column(*plots))